In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def save_lc(lc,filename):
    '''This function takes a lightcurve and saves it to the koi_data.txt file'''
    lc_data = np.array([lc[0],lc[1],lc[2]]).transpose()
    np.savetxt(filename,lc_data, fmt='%6f')

In [3]:
def fivecolumn_file_to_array(file):
    '''This function takes data from a txt file and returns the time, flux, and err columns as arrays'''
    time,input_flux,flux_err,output_flux,irrelevant = np.loadtxt(file,unpack=True)
    return time,input_flux,flux_err,output_flux

In [4]:
def threecolumn_file_to_array(file):
    '''This function takes data from a txt file and returns the time, flux, and err columns as arrays'''
    time,input_flux,flux_err = np.loadtxt(file,unpack=True)
    return time,input_flux,flux_err

In [5]:
def normalise(lc):
    peak = np.percentile(lc['flux'], 97)
    lc_norm = lc
    lc_norm['flux'] = lc['flux']/peak
    lc_norm['flux_err'] = lc['flux_err']/peak
    return lc_norm

In [6]:
def plot_model(file,i):
    stsp_output = fivecolumn_file_to_array(file)
    #og_data = threecolumn_file_to_array('Transit_15/koi_data_t15.txt')
    fig,ax = plt.subplots(figsize=(10,5))
    ax.plot(stsp_output[0],stsp_output[3],ls='-',lw=0.5,marker='.',ms=5,c='r',label='STSP Model')
    ax.plot(stsp_output[0],stsp_output[1],ls='-',lw=0.5,marker='.',ms=5,c='b',label='Original Data')
    #ax.plot(og_data[0],og_data[1],ls='-',lw=0.5,marker='.',ms=1,c='g',label='Original Unbinned Data')
    lgd = ax.legend(edgecolor='k', fontsize=18)
    #ax.set_title(transit_names[i-1], fontsize=18, fontweight='bold')
    ax.set_xlabel('Time (BJD - 245833 days)', fontsize=18)
    ax.set_ylabel('Relative Flux', fontsize=18)

In [7]:
def centre_transit(lc,midpoint,period):
    times = []
    fluxes = []
    fluxerrs = []
    for i in range(len(lc['time'])):
        times.append(lc['time'][i]) #.value)
    for i in range(len(lc['flux'])):
        fluxes.append(lc['flux'][i])
    for i in range(len(lc['flux_err'])):
        fluxerrs.append(lc['flux_err'][i])
    start_point = midpoint - period/2
    end_point = midpoint + period/2
    transit_time = []
    transit_flux = []
    transit_errs = []
    for i in range(len(times)):
        if start_point < times[i] < end_point:
            transit_time.append(times[i])
            transit_flux.append(fluxes[i])
            transit_errs.append(fluxerrs[i])
    centred_array = np.array([transit_time,transit_flux,transit_errs])
    return centred_array

In [8]:
def plot_transit(i):
    '''There are 46 transits in all of the Kepler data for KOI-340. This function plots the centred (i+1)th transit'''
    transit_data = centre_transit(koi_final_lc,transit_midpoints[i])
    start_point = transit_midpoints[i] - 12.962304/2
    end_point = transit_midpoints[i] + 12.962304/2
    title = transit_names[i]
    fig,ax = plt.subplots(figsize=(7,4))
    ax.plot(transit_data[0],transit_data[1],ls='-',lw=0.5,marker='o',ms=3,c='k')
    #ax.set_title(title,fontsize=18,fontweight='bold')
    ax.set_xlabel('Time (BJD - 245833 days)',fontsize=18,fontweight='bold')
    ax.set_ylabel('Relative Flux',fontsize=18,fontweight='bold')
    ax.set_xlim(start_point,end_point)

In [9]:
def parambest_to_total_spotted_area(transit):
    file = open('Transit_{}/koi340_actionU_ex_parambest.txt'.format(transit))
    file_list = []
    for i in file:
        file_list.append(i)
    num_spots = int((len(file_list) - 3) / 6)
    spot_radii = []
    for i in range(num_spots):
        spot_radii.append(float(file_list[i*3].strip()))
    each_spot_areas = []
    for i in range(len(spot_radii)):
        area = 2*np.pi*(1 - np.cos(np.arcsin(spot_radii[i])))
        each_spot_areas.append(area)
    total_spotted_area = sum(each_spot_areas)/(4*np.pi)
    
    return total_spotted_area

In [10]:
def parambest_to_intransit_spotted_area(transit):
    file = open('Transit_{}/koi340_actionU_ex_parambest.txt'.format(transit))
    num_intransit_spots = fixed_spot_nums[stsp_transits.index(transit)]
    file_list = []
    for i in file:
        file_list.append(i)
    spot_radii = []
    for i in range(num_intransit_spots):
        spot_radii.append(float(file_list[i*3].strip()))
    each_spot_areas = []
    for i in range(len(spot_radii)):
        area = 2*np.pi*(1 - np.cos(np.arcsin(spot_radii[i])))
        each_spot_areas.append(area)
    total_spotted_area = sum(each_spot_areas)/(4*np.pi)
    
    return total_spotted_area

In [11]:
def parambest_to_outoftransit_spotted_area(transit):
    file = open('Transit_{}/koi340_actionU_ex_parambest.txt'.format(transit))
    file_list = []
    for i in file:
        file_list.append(i)
    spot_radii = []
    num_outoftransit_spots = int((len(file_list) - 3) / 6) - fixed_spot_nums[stsp_transits.index(transit)]
    num_intransit_spots = fixed_spot_nums[stsp_transits.index(transit)]
    file_list_without_fixed_spots = file_list[(num_intransit_spots*3):]
    for i in range(num_outoftransit_spots):
        spot_radii.append(float(file_list_without_fixed_spots[i*3].strip()))
    each_spot_areas = []
    for i in range(len(spot_radii)):
        area = 2*np.pi*(1 - np.cos(np.arcsin(spot_radii[i])))
        each_spot_areas.append(area)
    total_spotted_area = sum(each_spot_areas)/(4*np.pi)
    
    return total_spotted_area